In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import pickle

class Data:
    def __init__(self):
        # Assign commonly used libraries and functions as class attributes for convenience
        self.pd = pd                         # Pandas for data manipulation and analysis
        self.plt = plt                       # Matplotlib for plotting graphs
        self.sns = sns                       # Seaborn for statistical data visualization
        self.train_test_split = train_test_split  # Scikit-learn function to split data into training and test sets
        self.StandardScaler = StandardScaler      # Scikit-learn class for feature scaling
        self.LabelEncoder = LabelEncoder          # Scikit-learn class for encoding categorical labels
        self.SVR = SVR                             # Support Vector Regression model from scikit-learn
        self.mean_squared_error = mean_squared_error  # Function to evaluate model performance (MSE)
        self.r2_score = r2_score                    # Function to evaluate model performance (R² score)
        self.pickle = pickle                        # Python module for serializing and saving models


Data Preprocessing: 
The DataPreprocessing class prepares raw data for machine learning by handling missingvalues, encoding categorical features, and scaling numerical data.

In [2]:
class DataPreprocessing(Data):
    def __init__(self, file_path):
        super().__init__()
        self.file_path = file_path
        self.df = None
        self.scaler = self.StandardScaler()
        self.read_data()

    # 1. Read data
    def read_data(self):
        self.df = self.pd.read_csv(self.file_path)
        print(f"✅ Data loaded successfully from: {self.file_path}")

    # 2. Rename columns
    def rename_columns(self):
        self.df.columns = [col.strip().title().replace("_", "") for col in self.df.columns]
        print("✅ Columns renamed successfully.")

    # 3. Show all columns
    def show_columns(self):
        print("📋 Columns in dataset:\n", self.df.columns.tolist())

    # 4. Show shape
    def show_shape(self):
        print("📊 Dataset shape:", self.df.shape)

    # 5. Show dataset info
    def show_info(self):
        print(self.df.info())

    # 6. Describe dataset
    def describe_data(self):
        print("📈 Dataset Description:\n", self.df.describe())

    # 7. Show min & max
    def show_min_max(self):
        print("Minimum values:\n", self.df.min())
        print("\nMaximum values:\n", self.df.max())

    # 8. Mean, Mode, Median
    def show_mean_mode_median(self, column):
        mean_val = self.df[column].mean()
        mode_val = self.df[column].mode()[0]
        median_val = self.df[column].median()
        print(f"Column: {column}")
        print(f"Mean: {mean_val}")
        print(f"Mode: {mode_val}")
        print(f"Median: {median_val}")

    # 9. Check null values
    def check_nulls(self):
        print("🧾 Null Values Summary:\n", self.df.isnull().sum())

    # 10. Handle missing values
    def handle_missing_values(self, strategy="fill"):
        if strategy == "fill":
            for col in self.df.columns:
                if self.df[col].isnull().sum() > 0:
                    if self.df[col].dtype == 'object':
                        self.df[col].fillna(self.df[col].mode()[0], inplace=True)
                    else:
                        self.df[col].fillna(self.df[col].mean(), inplace=True)
            print("✅ Missing values filled with mean/mode.")
        elif strategy == "drop":
            self.df.dropna(inplace=True)
            print("🗑️ Rows with missing values dropped.")

    # 11. Display data preview
    def basic_data_load(self, numbers=5, mode="head"):
        mode = mode.lower().strip()
        if mode == "head":
            print(self.df.head(numbers))
        elif mode == "tail":
            print(self.df.tail(numbers))
        elif mode == "sample":
            print(self.df.sample(numbers))
        else:
            print("❌ Invalid mode! Choose from: head, tail, sample.")


In [3]:
file_path = r"C:\Users\Administrator\Desktop\AI(LAB)\Task 10\data.csv"

# Step 1: Create object
data_processor = DataPreprocessing(file_path)

# Step 2: Rename columns
data_processor.rename_columns()

# Step 3: Show all columns
data_processor.show_columns()

# Step 4: Show dataset shape
data_processor.show_shape()

# Step 5: Show info about dataset
data_processor.show_info()

# Step 6: Describe data (statistics)
data_processor.describe_data()

# Step 7: Show min & max values
data_processor.show_min_max()

# Step 8: Show mean, mode, and median of 'Grades' (change if your column name differs)
data_processor.show_mean_mode_median(column="Grades")

# Step 9: Check null values
data_processor.check_nulls()

# Step 10: Handle missing values (choose 'drop' or 'fill')
data_processor.handle_missing_values(strategy="fill")

# Step 11: Show first 5 rows
data_processor.basic_data_load(numbers=5, mode="head")

# Step 12: Show last 10 rows
data_processor.basic_data_load(numbers=10, mode="tail")

# Step 13: Show random 7 rows
data_processor.basic_data_load(numbers=7, mode="sample")


✅ Data loaded successfully from: C:\Users\Administrator\Desktop\AI(LAB)\Task 10\data.csv
✅ Columns renamed successfully.
📋 Columns in dataset:
 ['StudentId', 'StudyHoursPerDay', 'ExtracurricularHoursPerDay', 'SleepHoursPerDay', 'SocialHoursPerDay', 'PhysicalActivityHoursPerDay', 'StressLevel', 'Gender', 'Grades']
📊 Dataset shape: (149, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   StudentId                    149 non-null    int64  
 1   StudyHoursPerDay             149 non-null    float64
 2   ExtracurricularHoursPerDay   149 non-null    float64
 3   SleepHoursPerDay             149 non-null    float64
 4   SocialHoursPerDay            149 non-null    float64
 5   PhysicalActivityHoursPerDay  149 non-null    float64
 6   StressLevel                  149 non-null    object 
 7   Gender                      

In [4]:
class Model(DataPreprocessing):
    def __init__(self, file_path):
        super().__init__(file_path)
        self.X = None
        self.y = None
        self.X_train = None
        self.X_test = None
        self.Y_train = None
        self.Y_test = None
        self.model = None

    # 1. Split into X and Y
    def split_xy(self, target_column="Grades"):
        self.X = self.df.drop(columns=[target_column])
        self.y = self.df[target_column]

        print("✅ X and Y split successfully.")
        print("X Shape:", self.X.shape)
        print("Y Shape:", self.y.shape)

    # 2. Encode categorical data (if any)
    def encode_categorical(self):
        cat_cols = self.X.select_dtypes(include=['object']).columns
        if len(cat_cols) > 0:
            for col in cat_cols:
                lbl = self.LabelEncoder()
                self.X[col] = lbl.fit_transform(self.X[col])

        print("🔠 Categorical columns encoded.")

    # 3. Train-test split
    def train_test_split_data(self, test_size=0.3):
        self.X_train, self.X_test, self.Y_train, self.Y_test = self.train_test_split(
            self.X, self.y, test_size=test_size, shuffle=False
        )
        print("📚 Train-test split done.")
        print("Training Data Shape:", self.X_train.shape)
        print("Testing Data Shape:", self.X_test.shape)

    # 4. Train SVR Model
    def train_model(self):
        self.model = self.SVR(kernel='rbf')
        self.model.fit(self.X_train, self.Y_train)
        print("🤖 SVR Model trained successfully.")

    # 5. Prediction
    def predict(self):
        predictions = self.model.predict(self.X_test)
        print("📈 Predictions:")
        print(predictions[:10])  # show first 10 predictions
        return predictions

    # 6. Display accuracy (R2 Score)
    def evaluate(self):
        predictions = self.model.predict(self.X_test)
        r2 = self.r2_score(self.Y_test, predictions)
        mse = self.mean_squared_error(self.Y_test, predictions)

        print("🎯 Model Performance:")
        print("R² Score:", r2)
        print("MSE:", mse)

        return r2, mse

    # 7. Save trained model
    def save_model(self, filename="svr_model.pkl"):
        with open(filename, "wb") as file:
            self.pickle.dump(self.model, file)
        print(f"💾 Model saved successfully as {filename}")


In [5]:
file_path = r"C:\Users\Administrator\Desktop\AI(LAB)\Task 10\data.csv"

# Create model object
model_obj = Model(file_path)

# Step 1: X and Y split
model_obj.split_xy(target_column="Grades")

# Step 2: Encode categorical columns
model_obj.encode_categorical()

# Step 3: Train-test split
model_obj.train_test_split_data(test_size=0.3)

# Step 4: Train SVR model
model_obj.train_model()

# Step 5: Predict
predictions = model_obj.predict()

# Step 6: Evaluate accuracy
model_obj.evaluate()

# Step 7: Save model
model_obj.save_model("SVR_Grade_Prediction.pkl")


✅ Data loaded successfully from: C:\Users\Administrator\Desktop\AI(LAB)\Task 10\data.csv
✅ X and Y split successfully.
X Shape: (149, 8)
Y Shape: (149,)
🔠 Categorical columns encoded.
📚 Train-test split done.
Training Data Shape: (104, 8)
Testing Data Shape: (45, 8)
🤖 SVR Model trained successfully.
📈 Predictions:
[7.61238333 7.49760069 7.49285239 7.56164413 7.64863733 7.53965658
 7.475638   7.50111044 7.48137198 7.38837324]
🎯 Model Performance:
R² Score: 0.019885685628429295
MSE: 0.4935331023512623
💾 Model saved successfully as SVR_Grade_Prediction.pkl
